In [201]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [202]:
%matplotlib inline

List of tresholds

In [203]:
lt = [i / 10 for i in range(1, 10)]

In [204]:
"""
l: list of distance values
td: Precision threshold
tc: Confidence threshold
"""
def precision(l, td, tc):
    n = len(l)
    num = 0
    den = 0
    for d, c in l:
        if c >= tc:
            den += 1
            if d <= td:
                num += 1
    if den != 0:
        return num / den

In [205]:
def recall(l, td, tc):
    n = len(l)
    num = 0
    for d, c in l:
        if c >= tc and d <= td:
            num += 1
    if (n != 0):
        return num / n

In [211]:
"""
l1: mf
l2: raw
"""
from IPython.display import HTML, display
import tabulate
def plotPrecision(l1, l2, lt, td, n):
    x = lt
    f1 = [0]*len(lt)
    f2 = [0]*len(lt)
    pl1 = [0]*len(lt)
    pl2 = [0]*len(lt)
    rl1 = [0]*len(lt)
    rl2 = [0]*len(lt)
    
    i = 0
    for t in lt:
        p1 = precision(l1, td, t)
        p2 = precision(l2, td, t)
        r1 = recall(l1, td, t)
        r2 = recall(l2, td, t)        
        pl1[i] = p1
        pl2[i] = p2
        rl1[i] = r1
        rl2[i] = r2
        
        f1[i] = 2*p1*r1/(p1+r1) if p1+r1 else 0 #F-measure of big file
        f2[i] = 2*p2*r2/(p2+r2) if p2+r2 else 0 #F-measure of yago raw file
        i += 1
        
    table = [lt,
             pl1,
             rl1,
             f1,
             pl2,
             rl2,
             f2]
    display(HTML(tabulate.tabulate(table, tablefmt='html')))
        
    df = pd.DataFrame({'x': x, 'ds_b': pl1, 'ds_a': pl2})
    colors = (200, 100, 0)
    area = np.pi*30
    
    plt.plot('x', 'ds_b', data=df, marker='o', markerfacecolor='blue', markersize=6, color='skyblue', linewidth=2)
    plt.plot('x', 'ds_a', data=df, marker="^", markerfacecolor='yellow', markersize=6, color='olive', linewidth=2)
    plt.legend()
    
    plt.title('')
    plt.xlabel('Confidence')
    plt.ylabel('Precision')
    plt.show()

In [213]:
import gzip as gz
import pickle as pkl

In [214]:
def check(arr):
    for x in arr:
        if not math.isnan(x):
            return True
    return False

In [215]:
f2 = gz.open('totalSimilarities_dbp_A.pkl.gz','rb')
draw = pkl.load(f2)
f3 = gz.open('totalSimilarities_dbp_B.pkl.gz','rb')
dmf = pkl.load(f3)

In [ ]:
"""
For 'setNum' choose one of the following options:
 1. only considers timestamp data
 2. considers results obtained from compute timestamp vs interval dates
 3. only considers interval data
setNum
"""
setNum = 3
x, conf1, _, _ = zip(*dmf[setNum])
data1 = np.array(x)
print("Rows: ", len(data1))
print("Cols: ", len(data1[0]))
r = 15
if setNum == 3:
    r = 18 
    temp = np.zeros((len(data1),18))
    temp[:,:-3] = data1
    for k in range(9,12):
        arr = np.array([data1[:,k], data1[:,k+3]])
        val = np.mean(arr, axis = 0, dtype=np.float64)
        temp[:,k+6] = val
    data1 = np.resize(data1,(len(data1),18))
    data1 = temp

x, conf2, _, _ = zip(*draw[setNum])
data2 = np.array(x)
print("Rows: ", len(data2))
print("Cols: ", len(data2[0]))
r = 15
if setNum == 3:
    r = 18 
    temp = np.zeros((len(data2),18))
    temp[:,:-3] = data2
    for k in range(9,12):
        arr = np.array([data2[:,k], data2[:,k+3]])
        val = np.mean(arr, axis = 0, dtype=np.float64)
        temp[:,k+6] = val
    data2 = np.resize(data2,(len(data2),18))
    data2 = temp

r = data1.shape[1]
for i in range(r):
    if check(data1[:,i]) and check(data2[:,i]):
        print("Graph", i)
        plotPrecision(np.column_stack((data1[:,i], conf1)), np.column_stack((data2[:,i], conf2)), lt, 0.1, 1000)